In [1]:
import time

import torch
from torch.optim.lr_scheduler import ReduceLROnPlateau

from torch_geometric.nn import VGAE
from torch_geometric.loader import DataLoader
from torch_geometric.utils import (degree, negative_sampling, 
                                   batched_negative_sampling)

from torch.utils.tensorboard import SummaryWriter

from gene_graph_dataset import G3MedianDataset
from phylognn_model import G3Median_GCNConv

In [2]:
gpuid = 0

train_p, test_p, val_p = 0.7, 0.2, 0.1
train_batch, test_batch, val_batch = 128, 256, 4

In [3]:
device = torch.device('cuda:' + str(gpuid) if torch.cuda.is_available() else 'cpu')

In [4]:
dataset = G3MedianDataset('dataset_g3m', 10, 10)

In [5]:
data_size = 1000 # len(dataset)
train_size, test_size, val_size = ((int)(data_size * train_p), 
                                   (int)(data_size * test_p), 
                                   (int)(data_size * val_p))

In [6]:
dataset = dataset.shuffle()
train_dataset = dataset[:train_size]
test_dataset = dataset[train_size:(train_size + test_size)]
val_dataset = dataset[(train_size + test_size):(train_size + test_size + val_size)]

In [7]:
test_dataset = list(test_dataset)
for t in test_dataset:
    t.neg_edge_label_index = negative_sampling(t.pos_edge_label_index, 
                                        t.num_nodes,
                                        t.num_nodes**2)
train_dataset = list(train_dataset)
for t in train_dataset:
    t.neg_edge_label_index = negative_sampling(t.pos_edge_label_index, 
                                        t.num_nodes,
                                        t.num_nodes**2)
val_dataset = list(val_dataset)
for t in val_dataset:
    t.neg_edge_label_index = negative_sampling(t.pos_edge_label_index, 
                                        t.num_nodes,
                                        t.num_nodes**2)

In [8]:
train_loader = DataLoader(train_dataset, batch_size = train_batch, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size = test_batch)
val_loader = DataLoader(val_dataset, batch_size= val_batch)

In [9]:
# deg = torch.zeros(5, dtype=torch.long)
# for data in train_dataset:
#     d = degree(data.edge_index[1].type(torch.int64), 
#                num_nodes=data.num_nodes, dtype=torch.long)
#     deg += torch.bincount(d, minlength=deg.numel())

In [10]:
in_channels, out_channels = dataset.num_features, 16

In [11]:
model = VGAE(G3Median_GCNConv(in_channels, out_channels)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=30,
                              min_lr=0.00001)

In [12]:
writer = SummaryWriter(log_dir='runs_g3m_10/g3median_lngtrain_gcn_aneg_run7')

In [13]:
from torch_geometric.data import Batch
def train(train_loader):
    model.train()
    
    total_loss = 0
    for data in train_loader:
        
        data = data.to(device)
        optimizer.zero_grad()
        
        z = model.encode(data.x, data.edge_index)
        loss = model.recon_loss(z, data.pos_edge_label_index, data.neg_edge_label_index)
        loss = loss + (1 / data.num_nodes) * model.kl_loss()
        loss.backward()
        optimizer.step()
        
        total_loss += loss
    return total_loss/len(train_loader)

In [14]:
@torch.no_grad()
def test(test_loader):
    model.eval()
    auc, ap = 0, 0
    
    for data in test_loader:
        
        data = data.to(device)
        
        z = model.encode(data.x, data.edge_index)
        # loss += model.recon_loss(z, data.pos_edge_label_index, data.neg_edge_label_index)
        tauc, tap = model.test(z, data.pos_edge_label_index, data.neg_edge_label_index)
        
        auc += tauc
        ap += tap
        
    return auc/len(test_loader), ap/len(test_loader)

In [15]:
@torch.no_grad()
def val(val_loader):
    model.eval()
    loss = 0
    
    for data in val_loader:
        
        data = data.to(device)
        
        z = model.encode(data.x, data.edge_index)
        loss += model.recon_loss(z, data.pos_edge_label_index, data.neg_edge_label_index)
        # tauc, tap = model.test(z, data.pos_edge_label_index, data.neg_edge_label_index)
                
    return loss/len(val_loader)

In [ ]:
for epoch in range(1, 1000 + 1):
    # print(f'{time.ctime()} - Epoch: {epoch:04d}')
    loss = train(train_loader)
    # print(f'{time.ctime()} - \t train loss: {loss:.6f}')
    tloss = val(val_loader)
    # print(f'{time.ctime()} - \t val   loss: {tloss:.6f}')
    # scheduler.step(tloss)
    
    writer.add_scalar('Loss/train', loss, epoch)
    writer.add_scalar('Loss/val', tloss, epoch)
    
    
    auc, ap = test(test_dataset)
    
    writer.add_scalar('AUC/test', auc, epoch)
    writer.add_scalar('AP/test', ap, epoch)
    
    if epoch % 50 == 0:
        print(f'{time.ctime()} - Epoch: {epoch:04d}        auc: {auc:.6f}, ap: {ap:.6f}')

Mon Dec 27 00:35:08 2021 - Epoch: 0010        auc: 0.792897, ap: 0.198137
Mon Dec 27 00:35:50 2021 - Epoch: 0020        auc: 0.841391, ap: 0.239281
Mon Dec 27 00:36:32 2021 - Epoch: 0030        auc: 0.855600, ap: 0.249919
Mon Dec 27 00:37:14 2021 - Epoch: 0040        auc: 0.857552, ap: 0.253432
Mon Dec 27 00:37:56 2021 - Epoch: 0050        auc: 0.866992, ap: 0.260883
Mon Dec 27 00:38:36 2021 - Epoch: 0060        auc: 0.870816, ap: 0.270183
Mon Dec 27 00:39:17 2021 - Epoch: 0070        auc: 0.868010, ap: 0.263493
Mon Dec 27 00:40:00 2021 - Epoch: 0080        auc: 0.875030, ap: 0.274623
Mon Dec 27 00:40:42 2021 - Epoch: 0090        auc: 0.877226, ap: 0.278507
Mon Dec 27 00:41:22 2021 - Epoch: 0100        auc: 0.878630, ap: 0.278207
Mon Dec 27 00:42:01 2021 - Epoch: 0110        auc: 0.879650, ap: 0.281811
Mon Dec 27 00:42:41 2021 - Epoch: 0120        auc: 0.878826, ap: 0.280316
Mon Dec 27 00:43:23 2021 - Epoch: 0130        auc: 0.886195, ap: 0.292040
Mon Dec 27 00:44:04 2021 - Epoch: 0140

In [ ]:
writer.close()

In [ ]:
# torch.save(model.state_dict(), 'g2g_test_model_batch')